Импорт источника данных с внешнего ресурса:

In [1]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': '-----', # имя пользователя
            'pwd': '-----', # пароль
            'host': '-----',
            'port': 1111, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

query = '''
            SELECT * FROM dash_visits
        '''

In [2]:
df = pd.io.sql.read_sql(query, con = engine) 

Прежде чем работать с данными дальше, проверим, что они корректны.

In [3]:
df.head(10)

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


В датафрейме представлена информация о действиях пользователей и событиях на Яндекс.Дзен:
- record_id - уникальный номер записи
- item_topic - тема карточки
- source_topic - тема источника
- age_segment - сегмент возраста пользователя
- dt - дата и время события
- visits - количество посещений.
Типы данных корректны, пропусков нет.

Дубликатов тоже нет:

In [5]:
df.duplicated().sum()

0

Id записей представляют собой числа от 1040597 до 1071341:

In [6]:
print(df['record_id'].min())
print(df['record_id'].max())

1040597
1071341


Все значения тем карточек корректны:

In [7]:
df['item_topic'].value_counts()

Отношения             1536
Интересные факты      1535
Наука                 1505
Подборки              1456
Полезные советы       1424
Общество              1422
Россия                1385
История               1363
Семья                 1287
Путешествия           1247
Деньги                1234
Женщины               1230
Дети                  1229
Туризм                1206
Здоровье              1203
Красота               1193
Культура              1160
Юмор                  1129
Искусство             1119
Рассказы              1109
Психология            1056
Скандалы              1023
Знаменитости           976
Женская психология     914
Шоу                    804
Name: item_topic, dtype: int64

Значения тем источников тоже:

In [8]:
df['source_topic'].value_counts()

Семейные отношения    1822
Россия                1687
Знаменитости          1650
Полезные советы       1578
Путешествия           1563
Кино                  1505
Дети                  1459
История               1437
Семья                 1405
Одежда                1379
Здоровье              1243
Искусство             1228
Авто                  1077
Психология            1055
Сад и дача            1036
Политика              1024
Спорт                 1007
Сделай сам             995
Ремонт                 985
Деньги                 973
Еда                    912
Интерьеры              809
Строительство          758
Музыка                 750
Технологии             741
Финансы                667
Name: source_topic, dtype: int64

В данных о возрастных сегметах аномалий нет:

In [9]:
df['age_segment'].value_counts()

18-25    7056
26-30    5875
31-35    5552
36-40    5105
41-45    3903
45+      3254
Name: age_segment, dtype: int64

Задан временной промежуток от 18:28 до 19:00 за 24 сентября 2019 г.

In [10]:
print(df['dt'].min())
print(df['dt'].max())

2019-09-24 18:28:00
2019-09-24 19:00:00


Переменная visits принимает значения от 1 до 371. Среднее значение - 10, медианное - 3.

In [11]:
df['visits'].describe()

count    30745.000000
mean        10.089673
std         19.727601
min          1.000000
25%          1.000000
50%          3.000000
75%         10.000000
max        371.000000
Name: visits, dtype: float64

Таким образом,аномалий в файлах не обнаружено. Запишем таблицу в файл формата .csv, с которым будем далее работать в Tableau:

In [12]:
df.to_csv(r'C:\project_automatization\dash_visits.csv', index = False)